# Capstone Project: 
# Identifying similar districts in different Peruvian provinces

## Introduction

Peru is a very varied country in culture and commerce. The objective of this project is to determine similarities between Peruvian districts of both the same province and between districts of different provinces. For this, the FourSquare venues data will be used as an indicator of commercial similarity to obtain clusters of different degrees of grouping, and compare the results with classifications made by national organizations such as INEI.

We will use the codes provided in the laboratories of the course to create functions for the purpose of modularizing the programming tasks.

The results of this project could be used to develop and implement government programs adapted to regions with similar needs.

## Data

We need the names of the districts of four selected provinces: Arequipa, Chincha, Cusco and Puno. This data can be extracted from Wikipedia. The districts of the districts will be combined with their respective provinces and country name (Peru in this case) to obtain the geolocation coordinates. These coordinates will be used to make venue requests in FourSquare.

Sources: 
* Wikipedia, https://www.wikipedia.org/
* Instituo Nacional de Estadística e Informática, https://www.inei.gob.pe/
